In [7]:
# Imports
import tensorflow as tf
from pathlib import Path

In [3]:
# Load Data Set
(x_train, y_train), (x_test, y_test) =tf.keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 14s 0us/step


In [4]:
# Normalize the data in 0 - 1 range
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255
x_test = x_test / 255

In [5]:
# Convert class vectors to binary class matrices 
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [17]:
# Create a model and add layers
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(32,32,3)))
model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2))) # Good to put a MaxPooling layer after each block of Convolutional Layers
model.add(tf.keras.layers.Dropout(0.25)) # Good to add after each MaxPooling layer

model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))# Good to put a MaxPooling layer after each block of Convolutional Layers
model.add(tf.keras.layers.Dropout(0.25))# Good to add after each MaxPooling layer

model.add(tf.keras.layers.Flatten()) # Should flatten when transitioning from Convolutional Layer to Dense Layer

model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.50)) # Good to add after a group of Dense Layers
model.add(tf.keras.layers.Dense(10, activation='softmax'))

# Compile the Model
model.compile(
    loss ='catigorical_crossentropy',
    optimizer =  'adam',
    metrics = ['accuracy']    
)

# Print a summary of the model
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 conv2d_13 (Conv2D)          (None, 30, 30, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 15, 15, 32)        0         
                                                                 
 conv2d_14 (Conv2D)          (None, 15, 15, 64)        18496     
                                                                 
 conv2d_15 (Conv2D)          (None, 13, 13, 64)        36928     
                                                      